# Classify mine from sonar data

# Libraries

In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [68]:
data=pd.read_csv('Sonar.csv')
data=data.sample(frac=1)
x=data.drop(['Class'],axis=1).values
y=pd.get_dummies([str(i) for i in data['Class']]).values
train_x,test_x,train_y,test_y= train_test_split(x,y,test_size=0.1)
train_x[1],train_y[1]

(array([0.01  , 0.0194, 0.0155, 0.0489, 0.0839, 0.1009, 0.1627, 0.2071,
        0.2696, 0.299 , 0.3242, 0.3565, 0.3951, 0.5201, 0.6953, 0.8468,
        1.    , 0.9278, 0.851 , 0.801 , 0.8142, 0.8825, 0.7302, 0.6107,
        0.7159, 0.8458, 0.6319, 0.4808, 0.6291, 0.7152, 0.6005, 0.4235,
        0.4106, 0.3992, 0.173 , 0.1975, 0.237 , 0.1339, 0.1583, 0.3151,
        0.1968, 0.2054, 0.1272, 0.1129, 0.1946, 0.2195, 0.193 , 0.1498,
        0.0773, 0.0196, 0.0122, 0.013 , 0.0073, 0.0077, 0.0075, 0.006 ,
        0.008 , 0.0019, 0.0053, 0.0019]), array([0, 1], dtype=uint8))

## Building ANN

In [157]:
class Neural_Network(object):
    
  def assign(self,x,y):
  #parameters
    self.inputSize = x
    self.hl1Size = 30
    self.hl2Size = 30
    self.outputSize = y

  #weights
    self.W1 = np.random.randn(self.inputSize, self.hl1Size) # (4*10) weight matrix from input to hidden layer 1
    self.W2 = np.random.randn(self.hl1Size,self.hl2Size) #(10*10) weight matrix for hidden layer 1 to hidden layer 2
    self.W3 = np.random.randn(self.hl2Size, self.outputSize) # (3x1) weight matrix from hidden layer 2 to output layer
    self.b1=np.random.randn(self.hl1Size)
    self.b2=np.random.randn(self.hl2Size)
    self.b3=np.random.randn(self.outputSize)

  def forward(self, X):
    #forward propagation through our network
    self.z = np.dot(X, self.W1)+self.b1 # dot product of X (input) and first set of weights and adding bias
    self.hl1 = self.sigmoid(self.z) # activation function
    
    self.z2 = np.dot(self.hl1, self.W2)+self.b2 # dot product of hidden layer1 and second set of weights and adding bias
    self.hl2 = self.sigmoid(self.z2) # activation function
    
    self.z3 = np.dot(self.hl2, self.W3)+self.b3 # dot product of hidden layer2 and third set of weights and adding bias
    o = self.sigmoid(self.z3) # final activation function
    
    return o

  def sigmoid(self, s):
    # activation function
    return 1/(1+np.exp(-s))

  def sigmoidPrime(self, s):
    #derivative of sigmoid
    return s * (1 - s)

  def backward(self, X, y, o):
    # backward propagate through the network
    self.o_error = y - o # error in output
    self.o_delta = self.o_error*self.sigmoidPrime(o)

    self.z3_error = self.o_delta.dot(self.W3.T)
    self.z3_delta = self.z3_error*self.sigmoidPrime(self.hl2) 
    
    self.z2_error = self.z3_delta.dot(self.W2.T) 
    self.z2_delta = self.z2_error*self.sigmoidPrime(self.hl1) 

    self.W1 += X.reshape(self.inputSize,1).dot(self.z2_delta.reshape(1,self.hl1Size)) # adjusting first set (input --> hidden) weights
    self.W2 += self.hl1.reshape(self.hl1Size,1).dot(self.z3_delta.reshape(1,self.hl2Size)) # adjusting second set (hidden --> output) weights
    self.W3 += self.hl2.reshape(self.hl2Size,1).dot(self.o_delta.reshape(1,self.outputSize)) # adjusting second set (hidden --> output) weights

  def train(self, X, y, n=1,batch=50):
    min,hw1,hw2,hw3,hb1,hb2,hb3=1,0,0,0,0,0,0
    self.assign(len(X[0]),len(y[0]))
    print("Before Training loss: "+str(np.mean(np.square(y - self.forward(X)))))
    for j in range(n):
        for k in range(0,len(X),batch):
            #if batch<len(X[k:]):
            #    batch=len(X[k:])
            avg_loss=[]
            for i in range(k,k+batch):
                o = self.forward(X[i])
                self.backward(X[i], y[i], o)
                l=np.mean(np.square(y - self.forward(X)))
                avg_loss.append(l)
                if min>l:
                    min=l
                    hw1,hb1=self.W1,self.b1
                    hw2,hb2=self.W2,self.b2
                    hw3,hb3=self.W3,self.b3
            #print("avg loss of this batch is " + str(np.mean(avg_loss)))
              
    print("min loss: "+str(min))
    self.W1,self.b1=hw1,hb1
    self.W2,self.b2=hw2,hb2
    self.W3,self.b3=hw3,hb3
    print("After Training loss: "+str(np.mean(np.square(y - self.forward(X)))))
   
  def predict(self,X):
    return self.forward(X).round()


In [158]:
NN=Neural_Network()
NN.train(train_x,train_y,n=100,batch=187)
prediction=NN.predict(test_x)
print("\naccuracy is "+str(accuracy_score(test_y, prediction)))

Before Training loss: 0.4087325057494607
min loss: 0.01081110966054334
After Training loss: 0.01081110966054334

accuracy is 0.8571428571428571
